# Model Rogue Rose

To Get your own API got to : [Open Router]("https://openrouter.ai/sophosympatheia/rogue-rose-103b-v0.2:free/api") , and choose the mpdel you want to test

## Installation 

```bash
    pip install pandas openai
```

In [2]:
import pandas as pd

#load the dataset
df = pd.read_csv("../original-data/helpdesk_dataset.csv")
df.head()

,user_query,intent,solution
0,"My password isn't working, can you help me res...",reset_password,Please follow the steps in the password reset ...
1,I can't log in; I think I forgot my password. ...,reset_password,A password reset link has been sent to your em...
2,I'm unable to access my account because my pas...,reset_password,Try resetting your password using the 'Forgot ...
3,Can you reset my password for me? I’ve been lo...,reset_password,Please check your email for a reset link and f...
4,I can't remember my password and need to reset...,reset_password,A reset link has been sent to your email. Use ...


## Test The API

In [2]:
api_key = "" # insert your api key

In [ ]:
from openai import OpenAI
import pandas as pd

# Initialize the OpenAI client with OpenRouter
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,  
)

def generate_variations_rogue(prompt, num_variations=5):
    """Generate variations of a query using Rogue Rose 103B v0.2 via OpenRouter"""
    variations = []
    for _ in range(num_variations):
        try:
            completion = client.chat.completions.create(
                model="sophosympatheia/rogue-rose-103b-v0.2:free",
                messages=[
                  {
                    "role": "user",
                    "content": prompt
                  }
                ],
                top_p=0.99,
                temperature=1.77,
                frequency_penalty=0.15,
                presence_penalty=1.1,
            )
            if completion.choices and completion.choices[0].message.content:
                variations.append(completion.choices[0].message.content.strip())
            else:
                print("Warning: Empty or invalid response from API")    
        except Exception as e:
            print(f"Error generating variations: {e}")
    return variations

In [5]:
# Example dataset
data = [
    {"user_query": "How do I reset my password?", "intent": "reset_password", "solution": "Please follow the steps in the password reset link sent to your email."},
    {"user_query": "I can't log in; I think I forgot my password.", "intent": "reset_password", "solution": "A password reset link has been sent to your email. Follow the instructions."},
    {"user_query": "Help me install the latest version of our video conferencing software.", "intent": "software_install", "solution": "Download the installer from the official site and follow the installation and setup instructions."}
]

# Generate synthetic data
synthetic_data = []
for row in data:
    prompt = f"""Generate 5 different ways to ask this technical support question: {row['user_query']}
        Important formatting rules:
            - Each question must end with a question mark
            - Questions should be written in a natural, conversational tone
            - Do not include any numbers, bullets, or prefixes
            - Separate questions only with question marks
            - Return the questions as a continuous text without line breaks

            Example format:
            What steps do I need to follow to reset my password? How can I go about resetting my password? Can you guide me on how to reset my password? What is the process to reset my password? How do I initiate a password reset?
            """
    variations = generate_variations_rogue(prompt, num_variations=2)
    for var in variations:
        # Split the phrases of var by question mark and clean them
        individual_phrases = [
            phrase.strip()
            for phrase in var.split('?')
            if phrase.strip()  # Remove empty phrases
            ]
        for phrase in individual_phrases:
            # Only add non-empty phrases
            if phrase:
                synthetic_data.append({
                    "user_query": phrase + "?",  # Add back the question mark
                    "intent": row["intent"],
                    "solution": row["solution"]
                })

# Convert to DataFrame
synthetic_df_test = pd.DataFrame(synthetic_data)
synthetic_df_test.head()

,user_query,intent,solution
0,1. What is the procedure to reset my password ...,reset_password,Please follow the steps in the password reset ...
1,"2. If I need to reset my password, what steps ...",reset_password,Please follow the steps in the password reset ...
2,3. Can you walk me through the process of rese...,reset_password,Please follow the steps in the password reset ...
3,4. What are the steps I need to follow to rese...,reset_password,Please follow the steps in the password reset ...
4,5. How can I initiate a password reset if I'm ...,reset_password,Please follow the steps in the password reset ...


### Function to clean bullet numbers

In [6]:
import re

def clean_numbered_bullets(df, column='user_query'):
    """
    Clean numbered bullets from the beginning and end of phrases in a DataFrame column.
    
    Args:
        df (pd.DataFrame): Input DataFrame
        column (str): Name of the column to clean
        
    Returns:
        pd.DataFrame: DataFrame with cleaned text
    """
    def clean_single_query(text):
        # Remove numbering patterns from the beginning (1., 2., etc.)
        text = re.sub(r'^\s*\d+\.\s*', '', text)
        
        # Remove quotation marks and numbers from the end
        text = re.sub(r'\s*"\d+"\s*$', '', text)
        
        # Remove any extra whitespace
        text = text.strip()
        
        return text
    
    # Apply the cleaning function to the specified column
    df[column] = df[column].apply(clean_single_query)
    
    return df

In [7]:
synthetic_df_test_copy = synthetic_df_test.copy()
clean_numbered_bullets(synthetic_df_test_copy)


,user_query,intent,solution
0,What is the procedure to reset my password if ...,reset_password,Please follow the steps in the password reset ...
1,"If I need to reset my password, what steps sho...",reset_password,Please follow the steps in the password reset ...
2,Can you walk me through the process of resetti...,reset_password,Please follow the steps in the password reset ...
3,What are the steps I need to follow to reset m...,reset_password,Please follow the steps in the password reset ...
4,How can I initiate a password reset if I'm una...,reset_password,Please follow the steps in the password reset ...
5,What is the procedure for resetting my password?,reset_password,Please follow the steps in the password reset ...
6,How can I go about changing my current passwor...,reset_password,Please follow the steps in the password reset ...
7,Can you provide me with the steps to reset my ...,reset_password,Please follow the steps in the password reset ...
8,"If I forgot my password, what should I do to r...",reset_password,Please follow the steps in the password reset ...
9,How do I initiate a password reset process for...,reset_password,Please follow the steps in the password reset ...


# Test On Own Data

In [8]:
# Generate synthetic data
synthetic_data = []
for index, row in df.iterrows():
    prompt = f"""Generate 5 different ways to ask this technical support question: {row['user_query']}
        Important formatting rules:
            - Each question must end with a question mark
            - Questions should be written in a natural, conversational tone
            - Do not include any numbers, bullets, or prefixes
            - Separate questions only with question marks
            - Return the questions as a continuous text without line breaks"""
    try:
        variations = generate_variations_rogue(prompt, num_variations=5)
        
        for var in variations:
            # Split the phrases of var by question mark and clean them
            individual_phrases = [
                phrase.strip()
                for phrase in var.split('?')
                if phrase.strip()  # Remove empty phrases
            ]
            
            for phrase in individual_phrases:
                # Only add non-empty phrases
                if phrase:
                    synthetic_data.append({
                        "user_query": phrase + "?",  # Add back the question mark
                        "intent": row["intent"],
                        "solution": row["solution"]
                    })
                        
    except Exception as e:
            print(f"Error processing row {index}: {e}")
            continue

# Convert to DataFrame
synthetic_df_rogue_rose = pd.DataFrame(synthetic_data)
synthetic_df_rogue_rose.head()

,user_query,intent,solution
0,"1. Hey there, I'm having trouble logging in. M...",reset_password,Please follow the steps in the password reset ...
1,"2. Hi, I'm having an issue with my account. I'...",reset_password,Please follow the steps in the password reset ...
2,"3. Hi, I'm having some trouble with my passwor...",reset_password,Please follow the steps in the password reset ...
3,"4. Hello, I'm having a problem with my account...",reset_password,Please follow the steps in the password reset ...
4,"5. Hi, I'm having difficulty logging in to my ...",reset_password,Please follow the steps in the password reset ...


In [9]:
synthetic_df_rogue_copy = synthetic_df_rogue_rose.copy()
synthetic_df_rogue_copy = clean_numbered_bullets(synthetic_df_rogue_copy)
synthetic_df_rogue_copy.head()

,user_query,intent,solution
0,"Hey there, I'm having trouble logging in. My p...",reset_password,Please follow the steps in the password reset ...
1,"Hi, I'm having an issue with my account. I'm s...",reset_password,Please follow the steps in the password reset ...
2,"Hi, I'm having some trouble with my password. ...",reset_password,Please follow the steps in the password reset ...
3,"Hello, I'm having a problem with my account lo...",reset_password,Please follow the steps in the password reset ...
4,"Hi, I'm having difficulty logging in to my acc...",reset_password,Please follow the steps in the password reset ...


In [10]:
synthetic_df_rogue_copy.to_csv("data/rogue_rose_data.csv", index=False)
len(synthetic_df_rogue_copy)

961

--------------------------------